In [1]:
!pip install "polars[all]"

   ---------------------------------------- 0.0/731.2 kB ? eta -:--:--
   ---------------------------------------- 731.2/731.2 kB 10.0 MB/s  0:00:00
   ---------------------------------------- 0.0/43.9 MB ? eta -:--:--
   -- ------------------------------------- 3.1/43.9 MB 15.4 MB/s eta 0:00:03
   ------ --------------------------------- 7.6/43.9 MB 20.4 MB/s eta 0:00:02
   --------- ------------------------------ 10.0/43.9 MB 16.3 MB/s eta 0:00:03
   ------------ --------------------------- 14.2/43.9 MB 16.8 MB/s eta 0:00:02
   ---------------- ----------------------- 18.1/43.9 MB 17.5 MB/s eta 0:00:02
   --------------------- ------------------ 23.1/43.9 MB 18.5 MB/s eta 0:00:02
   ------------------------- -------------- 27.8/43.9 MB 19.4 MB/s eta 0:00:01
   ----------------------------- ---------- 32.8/43.9 MB 19.8 MB/s eta 0:00:01
   ---------------------------------- ----- 37.7/43.9 MB 20.5 MB/s eta 0:00:01
   -------------------------------------- - 42.7/43.9 MB 20.9 MB/s eta 0

In [2]:
!wget https://github.com/akmtamimahmed/Lab-1.2/raw/refs/heads/main/data/lahman_1871-2024_csv/lahman_1871-2024_csv/Pitching.csv

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
!pip install more_polars


   ------------- -------------------------- 1/3 [composable]
   ---------------------------------------- 3/3 [more_polars]



In [4]:
import polars as pl
pl.Config.with_columns_kwargs = True

In [7]:
pitching = pl.read_csv('https://github.com/akmtamimahmed/Lab-1.2/raw/refs/heads/main/data/lahman_1871-2024_csv/lahman_1871-2024_csv/Pitching.csv')
pitching.head()

playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""aardsda01""",2004,1,"""SFN""","""NL""",1,0,11,0,0,0,0,32,20,8,1,10,5,0.417,6.75,0,0,2,0,61,5,8,0,1,1
"""aardsda01""",2006,1,"""CHN""","""NL""",3,0,45,0,0,0,0,159,41,24,9,28,49,0.214,4.08,0,1,1,0,225,9,25,1,3,2
"""aardsda01""",2007,1,"""CHA""","""AL""",2,1,25,0,0,0,0,97,39,23,4,17,36,0.3,6.4,3,2,1,0,151,7,24,2,1,1
"""aardsda01""",2008,1,"""BOS""","""AL""",4,2,47,0,0,0,0,146,49,30,4,35,49,0.268,5.55,2,3,5,0,228,7,32,3,2,4
"""aardsda01""",2009,1,"""SEA""","""AL""",3,6,73,0,0,0,38,214,49,20,4,34,80,0.19,2.52,3,2,0,0,296,53,23,2,1,2


In [9]:
try:
    df_era = (
        pl.scan_csv('https://github.com/akmtamimahmed/Lab-1.2/raw/refs/heads/main/data/lahman_1871-2024_csv/lahman_1871-2024_csv/Pitching.csv')
        .filter(pl.col('yearID') >= 1946)
        .group_by(['yearID', 'lgID'])
        .agg(
            pl.col('ER').sum().alias('total_ER'),
            pl.col('IPouts').sum().alias('total_IPouts')
        )
        .with_columns(
            ((pl.col('total_ER') * 27) / pl.col('total_IPouts')).alias('league_wide_ERA')
        )
        .with_columns(
            pl.when((pl.col("yearID") >= 2022) | (pl.col("yearID") == 2020))
            .then(pl.lit("Both_Leagues_DH"))
            .when((pl.col("yearID") >= 1973) & (pl.col("lgID") == "AL"))
            .then(pl.lit("AL_Only_DH"))
            .otherwise(pl.lit("No_DH"))
            .alias("dh_status")
        )
        .drop(['total_ER', 'total_IPouts'])
        .collect()
    )
    df_era.write_csv("league_era_dh_status.csv")
    print("Successfully wrote league_era_dh_status.csv")
    
except Exception as e:
    print(f"An error occurred during ERA analysis: {e}")

Successfully wrote league_era_dh_status.csv


In [10]:
print("\n--- Starting Slugging Percentage Analysis ---")


--- Starting Slugging Percentage Analysis ---


In [12]:
batting_url = pl.read_csv('https://github.com/akmtamimahmed/Lab-1.2/raw/main/data/lahman_1871-2024_csv/lahman_1871-2024_csv/Batting.csv')
batting_url.head()

playerID,yearID,stint,teamID,lgID,G,G_batting,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,G_old
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""aardsda01""",2004,1,"""SFN""","""NL""",11,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null
"""aardsda01""",2006,1,"""CHN""","""NL""",45,null,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,null
"""aardsda01""",2007,1,"""CHA""","""AL""",25,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null
"""aardsda01""",2008,1,"""BOS""","""AL""",47,null,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,null
"""aardsda01""",2009,1,"""SEA""","""AL""",73,null,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null


In [15]:
try:
    df_slg = (
        pl.scan_csv('https://github.com/akmtamimahmed/Lab-1.2/raw/main/data/lahman_1871-2024_csv/lahman_1871-2024_csv/Batting.csv')
        .filter(pl.col('yearID') >= 1946)
        .group_by(['yearID', 'lgID'])
        .agg(
            pl.col('H').sum().alias('total_H'),
            pl.col('2B').sum().alias('total_2B'),
            pl.col('3B').sum().alias('total_3B'),
            pl.col('HR').sum().alias('total_HR'),
            pl.col('AB').sum().alias('total_AB')
        )
        .with_columns(
            # Corrected: Use the aggregated columns ('total_H', etc.)
            pl.when(pl.col("total_AB") > 0)
            .then(
                (pl.col("total_H") + pl.col("total_2B") + (2 * pl.col("total_3B")) + (3 * pl.col("total_HR"))) / pl.col("total_AB")
            )
            .otherwise(pl.lit(0.0))
            .alias('league_wide_SLG')
        )
        .with_columns(
            pl.when((pl.col("yearID") >= 2022) | (pl.col("yearID") == 2020))
            .then(pl.lit("Both_Leagues_DH"))
            .when((pl.col("yearID") >= 1973) & (pl.col("lgID") == "AL"))
            .then(pl.lit("AL_Only_DH"))
            .otherwise(pl.lit("No_DH"))
            .alias("dh_status")
        )
        .drop(['total_H', 'total_2B', 'total_3B', 'total_HR', 'total_AB'])
        .collect()
    )
    df_slg.write_csv("league_slg_dh_status.csv")
    print("Successfully wrote league_slg_dh_status.csv")

except Exception as e:
    print(f"An error occurred during Slugging Percentage analysis: {e}")

Successfully wrote league_slg_dh_status.csv


In [16]:
print("\n--- Head of the ERA Analysis Table ---")
print(df_era.head())


--- Head of the ERA Analysis Table ---
shape: (5, 4)
┌────────┬──────┬─────────────────┬────────────┐
│ yearID ┆ lgID ┆ league_wide_ERA ┆ dh_status  │
│ ---    ┆ ---  ┆ ---             ┆ ---        │
│ i64    ┆ str  ┆ f64             ┆ str        │
╞════════╪══════╪═════════════════╪════════════╡
│ 1961   ┆ NL   ┆ 4.031227        ┆ No_DH      │
│ 2010   ┆ AL   ┆ 4.141416        ┆ AL_Only_DH │
│ 2003   ┆ AL   ┆ 4.529592        ┆ AL_Only_DH │
│ 2017   ┆ NL   ┆ 4.340542        ┆ No_DH      │
│ 1949   ┆ AL   ┆ 4.197697        ┆ No_DH      │
└────────┴──────┴─────────────────┴────────────┘


In [17]:
print("\n--- Head of the Slugging Percentage Analysis Table ---")
print(df_slg.head())


--- Head of the Slugging Percentage Analysis Table ---
shape: (5, 4)
┌────────┬──────┬─────────────────┬────────────┐
│ yearID ┆ lgID ┆ league_wide_SLG ┆ dh_status  │
│ ---    ┆ ---  ┆ ---             ┆ ---        │
│ i64    ┆ str  ┆ f64             ┆ str        │
╞════════╪══════╪═════════════════╪════════════╡
│ 1958   ┆ AL   ┆ 0.383097        ┆ No_DH      │
│ 1961   ┆ AL   ┆ 0.394744        ┆ No_DH      │
│ 1986   ┆ NL   ┆ 0.379994        ┆ No_DH      │
│ 1946   ┆ AL   ┆ 0.364142        ┆ No_DH      │
│ 1994   ┆ AL   ┆ 0.434056        ┆ AL_Only_DH │
└────────┴──────┴─────────────────┴────────────┘


In [18]:
print(f"\nLink to the data repository: https://github.com/akmtamimahmed/Lab-1.2/tree/main/data/lahman_1871-2024_csv/lahman_1871-2024_csv")


Link to the data repository: https://github.com/akmtamimahmed/Lab-1.2/tree/main/data/lahman_1871-2024_csv/lahman_1871-2024_csv
